In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from moviepy.editor import VideoClip, ImageClip, concatenate_videoclips
import pandas as pd
import math

runnersite = "https://runalyze.com/athlete/Schrottie"


In [ ]:
# Funktion zum Abrufen der Webseite und Extrahieren der relevanten Informationen
def extract_activity_data():
    response = requests.get(runnersite)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    activities = []
    today = datetime.today().strftime('%d.%m.%Y')
    
    # Finden aller Zeilen mit Laufaktivitäten
    activity_rows = soup.find_all('tr', class_='r')
    
    data = {
        'date': [],
        'a_type': [],
        'r_type': [],
        'distance': [],
        'duration': [],
        'pace': []
    }
    
    current_date = None
    
    for row in activity_rows:
        cols = row.find_all('td')
        
        if cols:
            date_cell = cols[1].text.strip()
            if date_cell == '': # komplett leer bedeutet, dass es nicht die erste Aktivität des Tages ist
                offset = 0
            else:
                offset = 1
                new_date = date_cell.split()[0] + '.' + str(datetime.today().year)
                # Das Datum noch "merken", damit es in Zeilen ohne Datum genutzt weden kann
                if new_date and new_date != current_date:
                    current_date = new_date
                    
            if row.find('i', class_='icons8-Running'):
                rt_str = cols[3 + offset].text.strip() if len(cols) > 3 + offset else ''
                distance_str = cols[4 + offset].text.strip().split()[0].replace(',', '.') if len(cols) > 4 + offset else ''
                duration_str = cols[5 + offset].text.strip() if len(cols) > 5 + offset else ''
                pace_str = cols[6 + offset].text.strip() if len(cols) > 6 + offset else ''
                
                # Überprüfen, ob die aktuellen Werte nicht leer sind
                if distance_str and duration_str and pace_str:
                    activities.append({'date': current_date, 'a_type': 'run', 'r_type': rt_str, 'distance': distance_str, 'duration': duration_str, 'pace': pace_str})
                else:
                    # Verwenden des vorherigen Datums, wenn die aktuellen Werte leer sind
                    if data['date']:
                        activities.append({'date': data['date'][-1], 'a_type': 'run', 'r_type': rt_str, 'distance': distance_str, 'duration': duration_str, 'pace': pace_str})
    
    df = pd.DataFrame(activities)
    df = df[df['a_type'] == 'run'] # Filtern nach Aktivitätstyp "run"
    # Dataframe aufräumen, damit Datentypen passen und später keine fehler bewirken
    #df['distance'] = df['distance'].str.replace(',', '').astype('int')
    df['distance'] = pd.to_numeric(df['distance'], errors='coerce')
    print(df)
    return df

In [ ]:
# Funktion zur Umwandlung der Dauer in Minuten
def convert_to_minutes(duration):
    time_format = "%H:%M:%S"
    duration_time = datetime.strptime(duration, time_format)
    return duration_time.hour * 60 + duration_time.minute + duration_time.second / 60

In [ ]:
# Funktion zur Erstellung des Diagramms
def create_chart(activities):
    # Anzahl der Zeilen im Dataframe
    n_rows = activities.shape[0]

    # Erstelle ein Figure-Objekt und ein Axes-Objekt
    fig, ax = plt.subplots(figsize=(10, 6))

    # Setze die X-Achsen-Beschriftungen
    ax.set_xlabel("Datum")
    # Erstelle zwei Y-Achsen, eine für Dauer und eine für Pace
    ax2 = ax.twinx()
    # Setze die Beschriftungen der Y-Achsen
    ax.set_ylabel("Distanz (km)")
    ax2.set_ylabel("Dauer (h:mm:ss) / Pace (min/km)")
    # Runde die maximalen Werte ab
    max_distance = math.ceil(activities["distance"].max())
    max_duration_minutes = math.ceil(activities["duration_minutes"].max())
    # Setze die Skalierung der Y-Achsen
    ax.set_ylim([0, max_distance * 1.1])
    ax2.set_ylim([0, max_duration_minutes * 1.1])

    # Zeichne die Balken für die Distanz und die Linien für die Dauer und Pace
    for i in range(n_rows):
        ax.bar(activities["date"].iloc[i], activities["distance"].iloc[i], label=activities["r_type"].iloc[i])
        ax2.plot([activities["date"].iloc[i], activities["date"].iloc[i]], 
                 [activities["duration_minutes"].iloc[i], activities["duration_minutes"].iloc[i]], 
                 color="C{}".format(i), label="Dauer")
        ax2.plot([activities["date"].iloc[i], activities["date"].iloc[i]], 
                 [activities["pace"].iloc[i], activities["pace"].iloc[i]], 
                 color="C{}".format(i), linestyle="dashed", label="Pace")

    # Füge eine Legende hinzu
    ax.legend(loc="upper left")
    ax2.legend(loc="upper right")

    # Zeige das Diagramm an
    plt.show()

# Erstelle ein Diagramm für jeden Datensatz
def create_multiple_charts(df):
    for i in range(1, len(df) + 1):
        create_chart(df.iloc[:i])

In [ ]:
# Funktion zur Erstellung des Videos
def create_video(activities):
    video_clips = []
    for activity in activities:
        text_clip = ImageClip('chart.png').set_duration(3).resize(height=720)
        text_clip = text_clip.set_pos(('center', 'top'))
        text_clip = text_clip.set_start(0)
        
        title_clip = ImageClip('chart.png').set_duration(3).resize(height=720)
        title_clip = title_clip.set_pos(('center', 'top'))
        title_clip = title_clip.set_start(3)
        
        video_clip = concatenate_videoclips([text_clip, title_clip])
        video_clips.append(video_clip)
    
    final_clip = concatenate_videoclips(video_clips)
    final_clip.write_videofile("weekly_activities.mp4", fps=24, codec='libx264')

In [ ]:
# Hauptfunktion
def main():
    activities = extract_activity_data()
    if not activities.empty:  # Check if activities is not empty
        #create_chart(activities)
        create_multiple_charts(activities)
        #create_video(activities)
        #print("Video wurde erfolgreich erstellt.")
    else:
        print("Keine Aktivitäten gefunden.")

if __name__ == "__main__":
    main()